## Callin Switzer
### 19 July 2018
### Image segmentation to build training set for deep NN

In [1]:
import cv2
cv2.__version__

'3.4.1'

In [2]:
import sys
print(sys.version)
print("Environment name: " + sys.executable.split("\\")[5]) # want opencv3

3.6.5 | packaged by conda-forge | (default, Apr  6 2018, 16:13:55) [MSC v.1900 64 bit (AMD64)]
Environment name: opencv3


In [3]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk
import time
import pandas as pd
import skimage.io as io
import os


In [4]:
vidPath = r"D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52_556\2018_02_27__14_04_55_680_cam1-0005.avi"
os.path.isfile(vidPath)

True

In [5]:
# read in info about video
cap = cv2.VideoCapture(vidPath, 0)


length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

print(length, width, height, fps)
cap.release()

546 1280 1024 10.0


In [6]:
def load_imgs(vidPath, start, stop, greyscale = True):
    cap = cv2.VideoCapture(vidPath)
    imgs = []
    for ff in range(length):
        ret, frame = cap.read()
        if ff <= start:
            continue
        
        if greyscale:
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        else:
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        if ff>start:
            imgs.append(img)
            
        if np.mod(ff, 10) == 0:
            print(ff)
        
        if (ff >= stop):
            break
            
    cap.release()
    return imgs

In [7]:
imgs = load_imgs(vidPath,300, 400, True)

310
320
330
340
350
360
370
380
390
400


In [8]:
# normalize between 0 and 1
i3 = imgs / np.max(imgs)

In [9]:
bkg = np.median(i3, axis=0)

In [10]:
plt.clf()
io.imshow(bkg)

In [11]:
bkgRemoved = [i3[ii,:,:] - bkg for ii in range(i3.shape[0])]

In [12]:
bkgRemoved = (bkgRemoved - (np.min(bkgRemoved)))

In [299]:
np.min(bkgRemoved)

0.0

In [300]:

plt.clf()
io.imshow(bkgRemoved[2])

C:\Users\calli\Anaconda3\envs\opencv3\lib\site-packages\skimage\io\_plugins\matplotlib_plugin.py:77: UserWarning: Low image data range; displaying image with stretched contrast.
  warn("Low image data range; displaying image with "
C:\Users\calli\Anaconda3\envs\opencv3\lib\site-packages\skimage\io\_plugins\matplotlib_plugin.py:80: UserWarning: Float image out of standard range; displaying image with stretched contrast.
  warn("Float image out of standard range; displaying "


In [302]:
ctr=0
while(1):
    
    cv2.imshow("image",bkgRemoved[ctr])
    k = cv2.waitKey(80) & 0xFF
    if k == 27:
        break
    ctr+=1
    print(ctr)
    if ctr >=len(bkgRemoved):
        break
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [303]:
med = np.median(bkgRemoved)


In [304]:
ii = 0

In [734]:
import matplotlib.cm as cm1
ii -=1
#plt.clf()
# io.imshow((abs(bkgRemoved[ii] - med) > 0.2)*1.0, cmap = cm.gray)

In [735]:
# threshold
bwImg = ((abs(bkgRemoved[ii] - med) > 0.2)*1.0).astype("uint8")

In [736]:
plt.clf()
io.imshow(bwImg)

C:\Users\calli\Anaconda3\envs\opencv3\lib\site-packages\skimage\io\_plugins\matplotlib_plugin.py:77: UserWarning: Low image data range; displaying image with stretched contrast.
  warn("Low image data range; displaying image with "


In [737]:
#opening and closing
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(20,20))#np.ones((50,50),np.uint8)
# erosion = cv2.erode(bwImg,kernel,iterations = 1)
opening = cv2.morphologyEx(bwImg, cv2.MORPH_CLOSE, kernel)
#opening = cv2.morphologyEx(bwImg, cv.MORPH_GRADIENT, kernel)
#opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10)))
# plt.clf()
# io.imshow(opening)

In [738]:
plt.clf()

io.imshow(opening)

C:\Users\calli\Anaconda3\envs\opencv3\lib\site-packages\skimage\io\_plugins\matplotlib_plugin.py:77: UserWarning: Low image data range; displaying image with stretched contrast.
  warn("Low image data range; displaying image with "


In [739]:

im2, contours, hierarchy = cv2.findContours(opening, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
jj = cv2.drawContours(bkgRemoved[ii], contours, -1, (0,0,0), 1)
#cv2.imshow("image", bkgRemoved[ii])

In [740]:
#find the biggest area
if len(contours) != 0:
    c = max(contours, key = cv2.contourArea)

    x,y,w,h = cv2.boundingRect(c)
    # draw the book contour (in green)
    cv2.rectangle(bkgRemoved[ii],(x,y),(x+w,y+h),(0,255,0),2)
    cv2.imshow("image", bkgRemoved[ii])

In [515]:
len(contours)

16

In [157]:
ctr=0
while(1):
    
    cv2.imshow("image",bkgRemoved[ctr]*1.0)
    k = cv2.waitKey(80) & 0xFF
    if k == 27:
        break
    ctr+=1
    print(ctr)
    if ctr >=len(bkgRemoved):
        break
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [142]:
thresh[0]*1.0

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])